In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('../scripts') # adjust the path based on actual location
from quantitative_analysis import StockAnalyzer


In [ ]:
# Load your CSV

msf_price_data = pd.read_csv('../src/data/yfinance_data/MSFT_historical_data.csv')
# Create analyzer object

msf_analyzer = StockAnalyzer(msf_price_data)
# Prepare data
msf_analyzer .prepare_data()


In [ ]:
msf_analyzer.add_technical_indicators()

In [ ]:
msf_analyzer.plot_indicators()

In [ ]:
msf_analyzer.plot_candlestick(
   
)

In [ ]:
msf_analyzer.plot_volume()